Подготовка среды

In [84]:
#!g1.1
%pip install pyarrow
%pip install transformers
%pip install datasets
%pip install accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' com

In [88]:
#!g1.1
import torch
import torchvision
from torch.utils.data import DataLoader, random_split
import numpy as np
import pandas as pd
import math
from transformers import AutoTokenizer, AutoModelForCausalLM, TextDataset, DataCollatorForLanguageModeling
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import AutoConfig
from transformers import utils
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset, Dataset
import logging
import os

Загрузка модели и токенайзера



In [90]:
#!g1.1
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2", bos_token="<s>", eos_token="<\s>", pad_token="<pad>", sep_token="<sep>", cache_dir="/model")
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2", cache_dir="/model")
model.resize_token_embeddings(len(tokenizer))

[Errno 13] Permission denied: '/model'


OSError: Can't load config for 'sberbank-ai/rugpt3large_based_on_gpt2'. Make sure that:

- 'sberbank-ai/rugpt3large_based_on_gpt2' is a correct model identifier listed on 'https://huggingface.co/models'
  (make sure 'sberbank-ai/rugpt3large_based_on_gpt2' is not a path to a local directory with something else, in that case)

- or 'sberbank-ai/rugpt3large_based_on_gpt2' is the correct path to a directory containing a config.json file



Загрузка датасета

In [78]:
#!g1.1
df = pd.read_csv("Films.csv", encoding='utf-16')
df['Text'] = "Название: " + df['Title'].map(str) + "<sep>" + "Описание: " + df['Description'].map(str)
for i in range(0, 5):
    df['Text'] = df['Text'].apply(lambda x: str(x).replace("  ", " ")[:-1])
df['Text'] = df['Text'].apply(lambda x: str(x).replace("\t", ""))

Подготовка датасета

In [87]:
#!g1.1
train_dataset = Dataset.from_pandas(df)
def add_special_token(seq):
    seq["Text"] = '<s>' + seq["Text"] + '<\s>'
    return seq
train_dataset = train_dataset.map(add_special_token)
train_dataset = train_dataset.remove_columns(['Title', 'Description'])
def encode(examples):
    return tokenizer(examples['Text'], padding='max_length', max_length=256, truncation=True, return_special_tokens_mask=True,)  
train_dataset = train_dataset.map(encode, batched=True, remove_columns=["Text"])
print(train_dataset[19634])
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "special_tokens_mask"])
print(train_dataset[19634])

  0%|          | 0/27037 [00:00<?, ?ex/s]

  0%|          | 0/28 [00:00<?, ?ba/s]

{'input_ids': [1, 49639, 2235, 30, 15317, 382, 528, 3822, 5687, 50259, 630, 17408, 30, 478, 45176, 382, 528, 3822, 5687, 910, 2197, 411, 1949, 6426, 5473, 14733, 16, 5975, 289, 8597, 13818, 40638, 1258, 18, 630, 1832, 286, 807, 890, 16, 906, 4424, 19452, 30439, 1533, 3750, 41244, 779, 282, 26908, 1212, 15992, 478, 630, 2314, 17, 7530, 931, 861, 25021, 292, 685, 4618, 5258, 536, 16, 374, 282, 2615, 575, 9518, 2204, 31270, 25898, 10941, 289, 519, 4319, 1326, 5702, 2059, 411, 3322, 22290, 3314, 282, 30681, 34171, 10521, 18, 365, 5137, 4256, 333, 7310, 29257, 8445, 27147, 507, 16, 3876, 5103, 42961, 1080, 5002, 296, 1919, 282, 34781, 755, 1207, 281, 2230, 1114, 339, 2776, 298, 282, 1281, 12086, 1045, 334, 34689, 3437, 16471, 625, 18, 789, 18, 1009, 3822, 5687, 30, 478, 36382, 13725, 1135, 910, 478, 20651, 742, 403, 7729, 388, 470, 289, 478, 45176, 343, 290, 24242, 547, 4598, 763, 861, 365, 14131, 528, 3822, 5687, 282, 3106, 48185, 27734, 16, 594, 2822, 15817, 28383, 651, 34062, 354, 5772, 

In [80]:
#!g1.1
print([tokenizer.decode(x) for x in train_dataset[19634]['input_ids']])

['<s>', 'Наз', 'вание', ':', ' Сказ', 'ки', ' Г', 'оф', 'мана', '<sep>', 'О', 'писание', ':', ' «', 'Сказ', 'ки', ' Г', 'оф', 'мана', '»,', ' одна', ' из', ' двух', ' роман', 'тических', ' опер', ',', ' последнее', ' и', ' незакон', 'ченное', ' сочинение', ' Ж', '.', 'О', 'фф', 'ен', 'ба', 'ха', ',', ' была', ' впервые', ' представлена', ' публике', ' 10', ' февраля', ' 1881', ' года', ' в', ' париж', 'ском', ' театре', ' «', 'О', 'пера', '-', 'Ком', 'ик', '».', ' Успе', 'х', ' был', ' настолько', ' шум', 'ным', ',', ' что', ' в', ' первый', ' же', ' сезон', ' опера', ' выдержала', ' 101', ' представление', ' и', ' до', ' сих', ' пор', ' остается', ' одной', ' из', ' самых', ' востребов', 'анных', ' в', ' мировом', ' реперту', 'аре', '.', ' В', ' основе', ' пяти', 'ак', 'тной', ' оперы', ' лежит', ' пье', 'са', ',', ' напис', 'анная', ' Жю', 'лем', ' Бар', 'б', 'ье', ' в', ' соав', 'тор', 'стве', ' с', ' Ми', 'шел', 'ем', ' Кар', 'ре', ' в', ' 18', '51', ' году', ' по', ' мотивам', ' т

Задание параметров обучения

In [89]:
#!g1.1
#pragma async
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True)
training_args = TrainingArguments(
    "/home/jupyter/mnt/s3/model-output",
    num_train_epochs=30,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=16,
    warmup_steps=10,
    weight_decay=0.05
)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset, 
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None)
)
trainer.train()

Using mask_token, but it is not set yet.


ValueError: This tokenizer does not have a mask token which is necessary for masked language modeling. You should pass `mlm=False` to train on causal language modeling instead.

Background cell scheduled. Waiting for foreground cells to finish commits
Preparing g1.1 instance...
g1.1 instance is ready, running task...
Task is done, waiting for foreground cells to finish...
Merging task result to the state


In [45]:
text = "<s>Название: Американский огузок\n"
input_ids = tokenizer.encode(text, return_tensors="pt")
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=2,
                        temperature=1.5,
                        top_p=0.9,
                        max_length=512
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

['<s>Название: Плохие парни\nОписание:  В центре сюжета фильма«Плохие парни»—необычная история, которая разворачивается впригородной деревушке, гдепроисходит множество загадочных инепредсказуемых событий. Вэтом таинственном месте живут иживут трое главных героев, каждый изних по-своему, новсе они—плохие парни. Вцентре сюжета фильма«Плохие парни»—необычная история, которая разворачивается впригородной деревушке, гдепроисходит множество загадочных инепредсказуемых событий. Вэтом таинственном месте живут иживут трое главных героев, каждый изних по-своему, новсе они—плохие парни. Вцентре сюжета картины«Плохие парни»—необычная история, которая разворачивается впригородной деревушке, гдепроисходит множество загадочных инепредсказуемых событий. Вэтом таинственном месте живут иживут трое главных героев, каждый изних по-своему, новсе они—плохие парни. Вцентре сюжета фильма«Плохие парни»—необычная история, которая разворачивается впригородной деревушке, гдепроисходит множество загадочных']


In [ ]:
#!g1.1
